# PUCPR - PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ
### CURSO: BIG DATA E INTELIGÊNCIA ANALÍTICA
### MATÉRIA: BANCO DE DADOS APLICADO À BIG DATA
 
 PROFESSOR: WELLINGTON RODRIGO MONTEIRO
 
 ALUNO: JUAN MANOEL MARINHO NASCIMENTO  E GUSTAVO DA SILVA ROSSI

-  Somativo 2

### Elabore as seguintes etapas:

### 1. Mapear cada arquivo como uma tabela dentro do Apache Spark SQL, importando cada um deles.
### 2. Ajustar os tipos de dado correspondentes entre as colunas dos arquivos e das tabelas.

In [1]:
import os 
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.7'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.7'

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Banco de dados Aplicado a Big Data").config("spark.some.config.option","some-value").getOrCreate()

In [20]:
from pyspark import SparkContext
sc = SparkContext

In [31]:
# load data usando csv e separador 

tbl_endereco = spark.read.option("header",True).option("sep",";").csv("/home/jovyan/base/address.csv")
tbl_city = spark.read.option("header",True).option("sep",";").csv("/home/jovyan/base/city.csv")
tbl_customer = spark.read.option("header",True).option("sep",";").csv("/home/jovyan/base/customer.csv")
tbl_film = spark.read.option("header",True).option("sep",";").csv("/home/jovyan/base/film.csv")
tbl_language = spark.read.option("header",True).option("sep",";").csv("/home/jovyan/base/language.csv")

# croamdp tabelas usando as views do SparkSQL
tbl_endereco.createOrReplaceTempView("tbl_endereco")
tbl_city.createOrReplaceTempView("tbl_city")
tbl_customer.createOrReplaceTempView("tbl_customer")
tbl_film.createOrReplaceTempView("tbl_film")
tbl_language.createOrReplaceTempView("tbl_language")


In [36]:
spark.sql("""select tbl_language.name from tbl_language""").show()

+--------+
|    name|
+--------+
| English|
| Italian|
|Japanese|
|Mandarin|
|  French|
|  German|
+--------+



## 3. Elaborar uma consulta para responder à seguinte pergunta:

#### Quantos filmes existem em cada língua/idioma?


In [40]:
spark.sql("""

select tbl_language.language_id, tbl_language.name, count(*) as filmesEmCadaIdioma
from tbl_language, tbl_film
where tbl_language.language_id = tbl_film.language_id
group by tbl_language.language_id, tbl_language.name
order by filmesEmCadaIdioma asc
limit 1

""").show()

+-----------+-------+------------------+
|language_id|   name|filmesEmCadaIdioma|
+-----------+-------+------------------+
|          1|English|              1000|
+-----------+-------+------------------+



### 4. Elaborar uma consulta para responder à seguinte pergunta:

### Quantos clientes moram na mesma cidade?


In [49]:
spark.sql("""

select 
    tbl_customer.customer_id , 
    tbl_customer.first_name,  
    tbl_endereco.address_id ,  
    tbl_city.city_id, 
    tbl_city.city, 
    count(*) as ClientesPorCidade
from 
    tbl_customer, 
    tbl_endereco, 
    tbl_city  
where 
    tbl_customer.address_id = tbl_endereco.address_id and tbl_endereco.address_id = tbl_city.city_id
    
group by 
    tbl_customer.customer_id, 
    tbl_customer.first_name, 
    tbl_endereco.address_id,  
    tbl_city.city_id, 
    tbl_city.city
    
order by tbl_customer.customer_id asc
limit 5


""").show()

+-----------+----------+----------+-------+---------+-----------------+
|customer_id|first_name|address_id|city_id|     city|ClientesPorCidade|
+-----------+----------+----------+-------+---------+-----------------+
|          1|      MARY|         5|      5|    Adana|                1|
|         10|   DOROTHY|        14|     14|al-Manama|                1|
|        100|     ROBIN|       104|    104|   Cavite|                1|
|        101|     PEGGY|       105|    105|  Cayenne|                1|
|        102|   CRYSTAL|       106|    106|   Celaya|                1|
+-----------+----------+----------+-------+---------+-----------------+

